In [16]:
# A person class for spoofing Google

# your phone tracks:
# steps
# heartrate
# current location
# activity type: sleep, wake, walking, driving

# it can make inferences about calories burned, 
# do address place lookups, and current establishments

# location is updated > 14x/hour
# ad domains
# camera
# microphone
# call log
# SMS messages

> Since the spoofer is kind of like the Other's "heart" and "eyes" in terms of the data it produces, the idea of it existing in the gallery is kind of spooky, in a good way!  Sort of like physical proof of the coldness and inaccuracy of the data collected.  I like how the first image looks very technical.  The fact that it's generating quantitative versions of "emotions" makes that really funny.  

> I've also been thinking about how this project might exist online as well.  Since it is a simulated person, having feeds for the "eyes" (the Google Earth images", the "heart" (moods/sleep/BPM), and whatever may correlate to the Amazon searches ("brain", maybe?) could be interesting as well!  

In [17]:
# import time
import logging
import json
from datetime import datetime
import csv,sys

import simplejson as json
from shapely.geometry import Point
from shapely.wkt import dumps, loads
from peewee import *

In [18]:
DEBUG = True
logger = logging.getLogger(__name__)

if DEBUG:
    mydb = SqliteDatabase(':memory:')
else:
    mydb = SqliteDatabase("other.db")
    
# TZ METHOD 1: Hardcode zones:
# from_zone = tz.gettz('UTC')
# to_zone = tz.gettz('America/New_York')

# Use modular math => 7 hours behind UTC = (-360*8) (-2880)
# x = (x - 2880) % 86400

In [19]:
heartratedata = "../data/heartrate.json"
# this is 24 hours of heart-rate sampled on average every 2.76 seconds
# time has the format "19:14:00" "%H:%M:%S"
# appears to be UTC/GMT! Do we wnt to transform to local time? YES

# [{
#   "time" : "08:00:07",
#   "value" : {
#     "bpm" : 78,
#     "confidence" : 2
#   }
# }

sleepdata = "../data/sleep.json"


timepointdata = "../data/locations.csv"
# Start Time,End Time,Name,Latitude,Longitude
# lat/lng has been transformed and spoofed for Northern CA
# ten days were compressed into one for variety. 

stepdata = "../data/steps_monday.json"
# datetime has the format "Tue 19:14:00" "%a %H:%M:%S"
# appears to be UTC/GMT

# [{
#   "dateTime" : "Sat 08:03:00",
#   "value" : "0"
# },

In [33]:
def gimmeFeelings(myday=int(datetime.today().day)):
    feels = ["💛","💜","💜","💛","💜","💜","💛",
        "💛","💛","💛","💛","💛","💜","💜",
        "💛","💜","💜","💛","💛","💜","💛",
        "💜","💛","💛","💜","💜","💛","💛",
        "💜","💛","💛","💛","💜","💛","💜"]
    sleeps = ["➖","〰️","➖","➖","➖","➖","〰️",
        "➖","〰️","➖","〰️","➖","➖","➖",
        "➖","➖","➖","〰️","➖","➖","➖",
        "➖","➖","➖","➖","〰️","➖","〰️",
        "➖","〰️","➖","➖","➖","➖","➖"]
    
    return feels[myday], sleeps[myday]

def gimmeSeconds(thetime="08:02:02", thefmt="%H:%M:%S", timeadjust=0):
    s = datetime.strptime(thetime, thefmt).second * 1
    ms = datetime.strptime(thetime, thefmt).minute * 60
    hs = datetime.strptime(thetime, thefmt).hour * 3600

    if (timeadjust != 0):
        # timeadjust is set in hours
        correction = (timeadjust*360)
        utcsecs = int(s+ms+hs)
        # LAX is 7 hours behind UTC, -2520 seconds
        localsecs = (utcsecs-correction)% 86400
    else:
        localsecs = int(s+ms+hs)

    return int(localsecs)

def gimmecurseconds():
    now = datetime.now()     # should be local time!
    secs_since_midnight = (now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    return int(secs_since_midnight)

def gimmeclosestkv(mykeys):
#     mykeys = set().union(*(d.keys() for d in alistofdicts))
    mykey = min(mykeys, key=lambda x:abs(x - gimmecurseconds() ))
    q = Heart.select().where(Heart.timestamp == int(mykey))
    for entry in q:
        mybpm = entry.bpm    

    return mykey,mybpm

def gimmecurrsteps(mykeys):
    mykey = min(mykeys, key=lambda x:abs(x - gimmecurseconds() ))
    q = Step.select().where(Step.timestamp == int(mykey))
    for entry in q:
        mysteps = entry.steps    

    return mykey,mysteps

def gimmeclosestplace():
#     mykeys = set().union(*(d.keys() for d in alistofdicts))
    # get the keys by querying the timepoints
    mykeys = []
    q = Place.select()
    for entry in q:
        mykeys.append(int(entry.timestamp))

    mykey = min(mykeys, key=lambda x:abs(x - gimmecurseconds() ))

    q = Place.select().where(Place.timestamp == int(mykey))
    for entry in q:
        myplce = entry.point

    return mykey,myplce

In [22]:
def createPersondb(mydb):
    try:
        other, created = Person.get_or_create(name='OTHER')
    except:
        mydb.create_tables([Person])
        other, created = Person.get_or_create(name='OTHER')

    
    print("Person table is ready and 'OTHER' was created", created)
    return other

In [23]:
# Create the HEART table.
# Run this ONLY ONCE IN PRODUCTION!

def createHeartdb(mydb):
    with open(heartratedata, 'r') as f:
        json_text = f.read()

    heartrate_list = json.loads(json_text)

    for rec in heartrate_list:
        secs = gimmeSeconds(thetime=rec['time'], thefmt="%H:%M:%S", timeadjust=-7)
        bpm = rec['value']['bpm']

        try:
            beats, created = Heart.get_or_create(actor=other, timestamp=secs, bpm=bpm)
        except:
            mydb.create_tables([Heart])
            beats = Heart.create(actor=other, timestamp=secs, bpm=bpm)
            beats.save()
        
    print("Heart table is ready and 'beats' was created", created)

In [24]:
# Create the Step table.
# Run this ONLY ONCE IN PRODUCTION!

# [{
#   "dateTime" : "Sat 08:03:00",
#   "value" : "0"
# },

# actor = ForeignKeyField(Person, backref='steps')
# steps = IntegerField()
# timestamp = IntegerField()

def createStepdb(mydb):
    with open(stepdata, 'r') as f:
        json_text = f.read()

    step_list = json.loads(json_text)

    for rec in step_list:
        secs = gimmeSeconds(rec['dateTime'], thefmt="%a %H:%M:%S", timeadjust=0)
        val = rec['value']

        try:
            steps, created = Step.get_or_create(actor=other, timestamp=secs, steps=val)
        except:
            mydb.create_tables([Step])
            steps = Step.create(actor=other, timestamp=secs, steps=val)
            steps.save()
        
    print("Step table is ready and 'steps' was created", created)

In [25]:
def createPlacedb(mydb):
    with open(timepointdata, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        # This skips the first row of the CSV file.
        next(csvreader)
        for row in csvreader:
            mypoint = Point(float(row[3]), float(row[4]))
            try:
                tp, created = Place.get_or_create(actor=other, timestamp=gimmeSeconds(thetime=row[0], timeadjust=-7), point=dumps(mypoint), mode="WALK")
            except:
                mydb.create_tables([Place])
                tp = Place.create(actor=other, timestamp=gimmeSeconds(thetime=row[0], timeadjust=-7), point=dumps(mypoint), mode="WALK")
                tp.save()

    print("Place table is ready and 'steps' was created", created)

In [26]:
# create the tables or populate them if necessary

other = createPersondb(mydb)
createHeartdb(mydb)
createStepdb(mydb)
createPlacedb(mydb)

Person table is ready and 'OTHER' was created True
Heart table is ready and 'beats' was created True
Step table is ready and 'steps' was created True
Place table is ready and 'steps' was created True


In [27]:
# You don't want to run these on every query!
heartrate_keylist = []
q = Heart.select(Heart.timestamp)
for t in q:
    heartrate_keylist.append( int(t.timestamp) )

# You don't want to run this on every query
step_keylist = []
q = Step.select(Step.timestamp)
for t in q:
    step_keylist.append( int(t.timestamp) )

In [28]:
# what's our heart rate?
print("secs since mid",gimmecurseconds(),"closest key, bpm",gimmeclosestkv(heartrate_keylist))

secs since mid 56351 closest key, bpm (56353, 69)


In [29]:
# what's our location?
print("secs since mid",gimmecurseconds(),"closest key, bpm",gimmeclosestplace())

secs since mid 56352 closest key, bpm (60675, 'POINT (37.9219837557683022 -122.5306205502329959)')


In [31]:
# How many steps did we take?
print("secs since mid",gimmecurseconds(),"closest key, steps",gimmecurrsteps(step_keylist))

secs since mid 57588 closest key, steps (58200, 0)


In [45]:
print(gimmeFeelings())

('💜', '➖')
